In [2]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres, spsolve
from scipy.sparse import eye, diags, kron, csr_matrix
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import AllenCahn_Solver as AC_Neu

In [15]:
importlib.reload(AC_Neu)

<module 'AllenCahn_Solver' from 'D:\\Anaconda\\Code\\EnSF\\AllenCahn\\AllenCahn_Solver.py'>

In [27]:
xa = -0.5 
xb = 0.5
ya = -0.5
yb = 0.5 
# T = 10
# Nt = 1000

# T = 20
# Nt = 2000

T = 50
Nt = 5000
eps = 0.01
Nx = 128
Ny = Nx

N = Nx

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny

h = hx
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)

xmid = (x[:-1]+x[1:])/2
ymid = (y[:-1]+y[1:])/2

dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

S = 2

cons = 0

gamma = 1
## for uniform time step
b0 = 3/(2*dt)
b1 = -1/(2*dt)

# U0 = np.random.uniform(-0.9, 0.9, size=Nx*Ny)

In [5]:
# scipy.io.savemat('RefU0_AllenC_NonConsMob_v1.mat', {'U0': U0})

In [28]:
data = scipy.io.loadmat('RefU0_AllenC_NonConsMob_v1.mat')

U0 = data['U0'].astype(np.float64)
U0 = U0.reshape(Nx*Ny, )

In [29]:
# main diagonal
Main = (-2)*np.ones(N)
Main[0] = -1
Main[-1] = -1
Main *= (1/h**2)

# super- and sub-diagonals
Up =  (1/(h**2))*np.ones(N-1)
Low = (1/(h**2))*np.ones(N-1)

# assemble
G = diags(
    diagonals=[Main, Up, Low],
    offsets=[0, 1, -1],
    shape=(N, N),
    format='csr'
)
# G = np.diag(Main)       \
#   + np.diag(Up,  k= 1) \
#   + np.diag(Low,  k=-1)

I = eye(N, format='csr')
Dh = kron(I, G, format='csr') + kron(G, I, format='csr')

# Idd = np.eye(N)
# Dh = np.kron(Idd, G)+np.kron(G, Idd)

In [30]:
noise = 2*np.random.randn(Nx*Ny)
U1 = AC_Neu.AllenCahn_Solver_1step_BDF1(U0, N, dt, eps, gamma, S, Dh, 3, noise)

In [31]:
RefSol = np.zeros((Nx*Ny, Nt+1))

RefSol[:, 0] += U0
RefSol[:, 1] += U1

In [32]:
for ii in range(1, Nt):
    print(f'step={ii}:')
    t1 = time.time() 
    noise = 0.5*np.random.randn(Nx*Ny)
    x = AC_Neu.AllenCahn_Solver_1step_BDF2(U1, U0, N, dt, eps, gamma, S, Dh, b0, b1, 3, noise)
    t2 = time.time()
    print(f'\t time = {t2 - t1:.4f}')
    
    U0 = U1.copy()
    U1 = x.copy()
    RefSol[:, ii+1] +=x

step=1:
	 time = 0.1891
step=2:
	 time = 0.2081
step=3:
	 time = 0.2151
step=4:
	 time = 0.2109
step=5:
	 time = 0.2081
step=6:
	 time = 0.2684
step=7:
	 time = 0.2072
step=8:
	 time = 0.2167
step=9:
	 time = 0.2143
step=10:
	 time = 0.2102
step=11:
	 time = 0.2072
step=12:
	 time = 0.2045
step=13:
	 time = 0.2091
step=14:
	 time = 0.2067
step=15:
	 time = 0.2153
step=16:
	 time = 0.2142
step=17:
	 time = 0.2142
step=18:
	 time = 0.2080
step=19:
	 time = 0.2058
step=20:
	 time = 0.2097
step=21:
	 time = 0.2063
step=22:
	 time = 0.2153
step=23:
	 time = 0.2111
step=24:
	 time = 0.2086
step=25:
	 time = 0.2056
step=26:
	 time = 0.2109
step=27:
	 time = 0.2074
step=28:
	 time = 0.2237
step=29:
	 time = 0.2090
step=30:
	 time = 0.2469
step=31:
	 time = 0.2106
step=32:
	 time = 0.2134
step=33:
	 time = 0.2107
step=34:
	 time = 0.2088
step=35:
	 time = 0.2092
step=36:
	 time = 0.2315
step=37:
	 time = 0.2205
step=38:
	 time = 0.2151
step=39:
	 time = 0.2153
step=40:
	 time = 0.2175
step=41:


	 time = 0.2022
step=321:
	 time = 0.2013
step=322:
	 time = 0.2092
step=323:
	 time = 0.1942
step=324:
	 time = 0.1994
step=325:
	 time = 0.1957
step=326:
	 time = 0.1985
step=327:
	 time = 0.1977
step=328:
	 time = 0.2020
step=329:
	 time = 0.1982
step=330:
	 time = 0.2011
step=331:
	 time = 0.1974
step=332:
	 time = 0.2005
step=333:
	 time = 0.1997
step=334:
	 time = 0.2062
step=335:
	 time = 0.2034
step=336:
	 time = 0.1995
step=337:
	 time = 0.2011
step=338:
	 time = 0.2045
step=339:
	 time = 0.2021
step=340:
	 time = 0.2021
step=341:
	 time = 0.1989
step=342:
	 time = 0.2044
step=343:
	 time = 0.1997
step=344:
	 time = 0.2080
step=345:
	 time = 0.1982
step=346:
	 time = 0.2091
step=347:
	 time = 0.2103
step=348:
	 time = 0.1996
step=349:
	 time = 0.2020
step=350:
	 time = 0.1981
step=351:
	 time = 0.2008
step=352:
	 time = 0.2023
step=353:
	 time = 0.2077
step=354:
	 time = 0.2000
step=355:
	 time = 0.2035
step=356:
	 time = 0.2061
step=357:
	 time = 0.2007
step=358:
	 time = 0.2

	 time = 0.2417
step=638:
	 time = 0.2211
step=639:
	 time = 0.2556
step=640:
	 time = 0.2563
step=641:
	 time = 0.2120
step=642:
	 time = 0.2516
step=643:
	 time = 0.2381
step=644:
	 time = 0.2171
step=645:
	 time = 0.2367
step=646:
	 time = 0.2317
step=647:
	 time = 0.2627
step=648:
	 time = 0.2071
step=649:
	 time = 0.2351
step=650:
	 time = 0.1957
step=651:
	 time = 0.2250
step=652:
	 time = 0.2336
step=653:
	 time = 0.2157
step=654:
	 time = 0.2061
step=655:
	 time = 0.2856
step=656:
	 time = 0.1967
step=657:
	 time = 0.2261
step=658:
	 time = 0.2221
step=659:
	 time = 0.2079
step=660:
	 time = 0.2112
step=661:
	 time = 0.2281
step=662:
	 time = 0.2131
step=663:
	 time = 0.2318
step=664:
	 time = 0.1896
step=665:
	 time = 0.1960
step=666:
	 time = 0.2110
step=667:
	 time = 0.1952
step=668:
	 time = 0.2467
step=669:
	 time = 0.2201
step=670:
	 time = 0.2256
step=671:
	 time = 0.2321
step=672:
	 time = 0.2101
step=673:
	 time = 0.2191
step=674:
	 time = 0.2078
step=675:
	 time = 0.2

	 time = 0.2261
step=954:
	 time = 0.2412
step=955:
	 time = 0.2130
step=956:
	 time = 0.2116
step=957:
	 time = 0.2467
step=958:
	 time = 0.2230
step=959:
	 time = 0.2100
step=960:
	 time = 0.2248
step=961:
	 time = 0.1931
step=962:
	 time = 0.2088
step=963:
	 time = 0.2779
step=964:
	 time = 0.2113
step=965:
	 time = 0.2301
step=966:
	 time = 0.1981
step=967:
	 time = 0.2322
step=968:
	 time = 0.2061
step=969:
	 time = 0.3391
step=970:
	 time = 0.2499
step=971:
	 time = 0.2836
step=972:
	 time = 0.2481
step=973:
	 time = 0.2324
step=974:
	 time = 0.2206
step=975:
	 time = 0.2481
step=976:
	 time = 0.2182
step=977:
	 time = 0.2021
step=978:
	 time = 0.2200
step=979:
	 time = 0.2251
step=980:
	 time = 0.2030
step=981:
	 time = 0.2030
step=982:
	 time = 0.2227
step=983:
	 time = 0.2129
step=984:
	 time = 0.2682
step=985:
	 time = 0.2151
step=986:
	 time = 0.2273
step=987:
	 time = 0.2222
step=988:
	 time = 0.2711
step=989:
	 time = 0.2219
step=990:
	 time = 0.2065
step=991:
	 time = 0.2

	 time = 0.2331
step=1260:
	 time = 0.2231
step=1261:
	 time = 0.2186
step=1262:
	 time = 0.2211
step=1263:
	 time = 0.2186
step=1264:
	 time = 0.2074
step=1265:
	 time = 0.2389
step=1266:
	 time = 0.2226
step=1267:
	 time = 0.2330
step=1268:
	 time = 0.2131
step=1269:
	 time = 0.2406
step=1270:
	 time = 0.2183
step=1271:
	 time = 0.2307
step=1272:
	 time = 0.2546
step=1273:
	 time = 0.2210
step=1274:
	 time = 0.2088
step=1275:
	 time = 0.2119
step=1276:
	 time = 0.2110
step=1277:
	 time = 0.2116
step=1278:
	 time = 0.1964
step=1279:
	 time = 0.2190
step=1280:
	 time = 0.2167
step=1281:
	 time = 0.2101
step=1282:
	 time = 0.2157
step=1283:
	 time = 0.2091
step=1284:
	 time = 0.2200
step=1285:
	 time = 0.2264
step=1286:
	 time = 0.2086
step=1287:
	 time = 0.2320
step=1288:
	 time = 0.2466
step=1289:
	 time = 0.2127
step=1290:
	 time = 0.2261
step=1291:
	 time = 0.2294
step=1292:
	 time = 0.2161
step=1293:
	 time = 0.2234
step=1294:
	 time = 0.2221
step=1295:
	 time = 0.1965
step=1296:
	

...

	 time = 0.2119
step=3696:
	 time = 0.1961
step=3697:
	 time = 0.2129
step=3698:
	 time = 0.2077
step=3699:
	 time = 0.2041
step=3700:
	 time = 0.1939
step=3701:
	 time = 0.2005
step=3702:
	 time = 0.2053
step=3703:
	 time = 0.2011
step=3704:
	 time = 0.2019
step=3705:
	 time = 0.2052
step=3706:
	 time = 0.2021
step=3707:
	 time = 0.2001
step=3708:
	 time = 0.2138
step=3709:
	 time = 0.2080
step=3710:
	 time = 0.1991
step=3711:
	 time = 0.1937
step=3712:
	 time = 0.2063
step=3713:
	 time = 0.2088
step=3714:
	 time = 0.2042
step=3715:
	 time = 0.2007
step=3716:
	 time = 0.2077
step=3717:
	 time = 0.2003
step=3718:
	 time = 0.2040
step=3719:
	 time = 0.2012
step=3720:
	 time = 0.2017
step=3721:
	 time = 0.2062
step=3722:
	 time = 0.2022
step=3723:
	 time = 0.1976
step=3724:
	 time = 0.1948
step=3725:
	 time = 0.2149
step=3726:
	 time = 0.2032
step=3727:
	 time = 0.2026
step=3728:
	 time = 0.2081
step=3729:
	 time = 0.1991
step=3730:
	 time = 0.2047
step=3731:
	 time = 0.2118
step=3732:
	

	 time = 0.2081
step=4001:
	 time = 0.2044
step=4002:
	 time = 0.2045
step=4003:
	 time = 0.1972
step=4004:
	 time = 0.1986
step=4005:
	 time = 0.1978
step=4006:
	 time = 0.1994
step=4007:
	 time = 0.1985
step=4008:
	 time = 0.2014
step=4009:
	 time = 0.1996
step=4010:
	 time = 0.2002
step=4011:
	 time = 0.1977
step=4012:
	 time = 0.2132
step=4013:
	 time = 0.1907
step=4014:
	 time = 0.1924
step=4015:
	 time = 0.1973
step=4016:
	 time = 0.1972
step=4017:
	 time = 0.2079
step=4018:
	 time = 0.2083
step=4019:
	 time = 0.2098
step=4020:
	 time = 0.1950
step=4021:
	 time = 0.2058
step=4022:
	 time = 0.1859
step=4023:
	 time = 0.2107
step=4024:
	 time = 0.1996
step=4025:
	 time = 0.2052
step=4026:
	 time = 0.1982
step=4027:
	 time = 0.2062
step=4028:
	 time = 0.2036
step=4029:
	 time = 0.2061
step=4030:
	 time = 0.2090
step=4031:
	 time = 0.2073
step=4032:
	 time = 0.2003
step=4033:
	 time = 0.2245
step=4034:
	 time = 0.2092
step=4035:
	 time = 0.2093
step=4036:
	 time = 0.2037
step=4037:
	

	 time = 0.2296
step=4305:
	 time = 0.1992
step=4306:
	 time = 0.1967
step=4307:
	 time = 0.1995
step=4308:
	 time = 0.1994
step=4309:
	 time = 0.1963
step=4310:
	 time = 0.1987
step=4311:
	 time = 0.1977
step=4312:
	 time = 0.1941
step=4313:
	 time = 0.2016
step=4314:
	 time = 0.1953
step=4315:
	 time = 0.2008
step=4316:
	 time = 0.2021
step=4317:
	 time = 0.1991
step=4318:
	 time = 0.2006
step=4319:
	 time = 0.2009
step=4320:
	 time = 0.2003
step=4321:
	 time = 0.1973
step=4322:
	 time = 0.1976
step=4323:
	 time = 0.1978
step=4324:
	 time = 0.2038
step=4325:
	 time = 0.2099
step=4326:
	 time = 0.2006
step=4327:
	 time = 0.1966
step=4328:
	 time = 0.2004
step=4329:
	 time = 0.2014
step=4330:
	 time = 0.2020
step=4331:
	 time = 0.2053
step=4332:
	 time = 0.2044
step=4333:
	 time = 0.1972
step=4334:
	 time = 0.1960
step=4335:
	 time = 0.2033
step=4336:
	 time = 0.1923
step=4337:
	 time = 0.2044
step=4338:
	 time = 0.1971
step=4339:
	 time = 0.2035
step=4340:
	 time = 0.1875
step=4341:
	

	 time = 0.2376
step=4610:
	 time = 0.2211
step=4611:
	 time = 0.2309
step=4612:
	 time = 0.2236
step=4613:
	 time = 0.2108
step=4614:
	 time = 0.2114
step=4615:
	 time = 0.1989
step=4616:
	 time = 0.2251
step=4617:
	 time = 0.2070
step=4618:
	 time = 0.2131
step=4619:
	 time = 0.2277
step=4620:
	 time = 0.2246
step=4621:
	 time = 0.2106
step=4622:
	 time = 0.2118
step=4623:
	 time = 0.2146
step=4624:
	 time = 0.2169
step=4625:
	 time = 0.2522
step=4626:
	 time = 0.2272
step=4627:
	 time = 0.2143
step=4628:
	 time = 0.2148
step=4629:
	 time = 0.2167
step=4630:
	 time = 0.2184
step=4631:
	 time = 0.2084
step=4632:
	 time = 0.1907
step=4633:
	 time = 0.2122
step=4634:
	 time = 0.2058
step=4635:
	 time = 0.2732
step=4636:
	 time = 0.2131
step=4637:
	 time = 0.2141
step=4638:
	 time = 0.2126
step=4639:
	 time = 0.2282
step=4640:
	 time = 0.2176
step=4641:
	 time = 0.2049
step=4642:
	 time = 0.2444
step=4643:
	 time = 0.2071
step=4644:
	 time = 0.2242
step=4645:
	 time = 0.1934
step=4646:
	

	 time = 0.2152
step=4914:
	 time = 0.2072
step=4915:
	 time = 0.1481
step=4916:
	 time = 0.1853
step=4917:
	 time = 0.2081
step=4918:
	 time = 0.2216
step=4919:
	 time = 0.1970
step=4920:
	 time = 0.2086
step=4921:
	 time = 0.1726
step=4922:
	 time = 0.2164
step=4923:
	 time = 0.2105
step=4924:
	 time = 0.2052
step=4925:
	 time = 0.2028
step=4926:
	 time = 0.1975
step=4927:
	 time = 0.1897
step=4928:
	 time = 0.1746
step=4929:
	 time = 0.1924
step=4930:
	 time = 0.2020
step=4931:
	 time = 0.2210
step=4932:
	 time = 0.2086
step=4933:
	 time = 0.1866
step=4934:
	 time = 0.1611
step=4935:
	 time = 0.1891
step=4936:
	 time = 0.1935
step=4937:
	 time = 0.2227
step=4938:
	 time = 0.1831
step=4939:
	 time = 0.2547
step=4940:
	 time = 0.2117
step=4941:
	 time = 0.1974
step=4942:
	 time = 0.2130
step=4943:
	 time = 0.1989
step=4944:
	 time = 0.2132
step=4945:
	 time = 0.2189
step=4946:
	 time = 0.2102
step=4947:
	 time = 0.2267
step=4948:
	 time = 0.2046
step=4949:
	 time = 0.2051
step=4950:
	

In [33]:
scipy.io.savemat('RefSol_AllenC_T10_NonConsMob_v1.mat', {'RefU':RefSol})

# scipy.io.savemat('RefSol_AllenC_T20_NonConsMob_v1.mat', {'RefU':RefSol})

# scipy.io.savemat('RefSol_AllenC_T50_NonConsMob_v1.mat', {'RefU':RefSol})